zero-shot-react-description: 此代理使用ReAct框架，仅基于工具的描述来确定要使用的工具。

1. 可以提供任意数量的工具。
2. 此代理需要为每个工具提供描述

测试样例: 谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少？

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.1)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

agent_executor.run('谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少？')

In [ ]:
print(agent.agent.llm_chain.prompt.template)

conversational-react-description: 此代理用于对话环境中，提示设计旨在使代理程序有助于对话，它使用ReAct框架来决定使用哪个工具，并使用内存来记忆历史对话。

In [ ]:
from langchain.tools import BaseTool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

class EvaluateMathExpression(BaseTool):
    name = "Math Evaluation"
    description = 'use this tool to evaluate a math expression.'

    def _run(self, input: str):
        return LLMChain(llm=llm, prompt=PromptTemplate(template=input, input_variables=[]))
    
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

In [ ]:
tools = [EvaluateMathExpression()] + load_tools(["serpapi"])

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3, # agent at taking a certain number of steps, this can be useful to ensure that they do not go haywire and take too many steps.
    early_stopping_method='generate', # By default, the early stopping uses method "force" which just returns that constant string. Alternatively, you could specify method generate which then does one FINAL pass through the LLM to generate an output.
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [ ]:
print(agent.agent.llm_chain.prompt.template)

In [ ]:
agent("谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少")

In [ ]:
agent("谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少, 必须使用tools去查找名字，年龄和计算")

In [ ]:
agent("你上一次的回答错了，你没有正确的找到莱昂纳多·迪卡普里奥的女朋友的年龄，你必须使用tools去查找年龄再重新计算最终结果")

In [ ]:
from langchain.agents.conversational.prompt import PREFIX, FORMAT_INSTRUCTIONS

prefix = """
Answer the following questions as best you can. You have access to the following tools:
TOOLS:
------

Assistant has access to the following tools and it prioritizes using tools to solve problems:"""

format_instructions = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

If you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""

In [ ]:
new_prompt = agent.agent.create_prompt(
    prefix=prefix,
    format_instructions=format_instructions,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt
print(agent.agent.llm_chain.prompt.template)

In [ ]:
agent("谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少")

In [ ]:
agent("谁是莱昂纳多·迪卡普里奥的女朋友？她当前的年龄的2次方是多少, 你不能用聊天记录的信息")

In [ ]:
agent.memory.buffer